## Calculate the Variance of each term

In [1]:
import matplotlib.pyplot as plt
import xarray as xr
import netCDF4
import numpy as np
from matplotlib import cm,colors
import scipy.signal as sgl
from scipy.stats import pearsonr
import os

In [2]:
USER = os.environ['USER']

### Phi = Phi_bar + Phi_O2 + Phi_T + Phi_S + Phi_resi
### Phi_O2 = Phi_O2sol - Phi_AOU

In [3]:
datadir = f'/glade/scratch/{USER}/DPLE-results'
filename = f'{datadir}/DPLE_driftcorrected_Phi_vertical_mean_ens_mean_monthly_to_yearly.nc'
ds0 = xr.open_dataset(filename)
filename = f'{datadir}/DPLE_driftcorrected_Phi_mean_dimension_reduced_vertical_mean_ens_mean_monthly_to_yearly.nc'
ds1 = xr.open_dataset(filename)
filename = f'{datadir}/DPLE_driftcorrected_Phi_components_O2_vertical_mean_ens_mean_monthly_to_yearly.nc'
ds2 = xr.open_dataset(filename)
filename = f'{datadir}/DPLE_driftcorrected_Phi_components_TEMP_vertical_mean_ens_mean_monthly_to_yearly.nc'
ds3 = xr.open_dataset(filename)
filename = f'{datadir}/DPLE_driftcorrected_Phi_components_SALT_vertical_mean_ens_mean_monthly_to_yearly.nc'
ds4 = xr.open_dataset(filename)
filename = f'{datadir}/DPLE_driftcorrected_Phi_components_AOU_vertical_mean_ens_mean_monthly_to_yearly.nc'
ds5 = xr.open_dataset(filename)

In [10]:
#layer = '0-200m'
layer = '200-600m'

In [11]:
a1, a2, b1, b2 = [240, 380, 125, 320]
# time period for evaluation 1954-2007
yr1 = 1954
yr2 = 2007
#
nterms = 20
nlead = 10
variance = np.ma.zeros([3, nterms, nlead, a2-a1, b2-b1])
variance.mask=True
#
for cc in range(3):
    Phi_ori = ds0[f'Phi_{cc+3}_{layer}'].isel(Y=range(0,54)).values      # 54,10,nlat,nlon
    Phi_bar = ds1[f'Phi_{cc}_{layer}'].values                            #    10,nlat,nlon
    Phi_O2 = ds2[f'Phi_O2_{cc}_{layer}'].isel(Y=range(0,54)).values      # 54,10,nlat,nlon
    Phi_T   = ds3[f'Phi_TEMP_{cc}_{layer}'].isel(Y=range(0,54)).values   # 54,10,nlat,nlon
    Phi_S   = ds4[f'Phi_SALT_{cc}_{layer}'].isel(Y=range(0,54)).values   # 54,10,nlat,nlon
    Phi_AOU = ds5[f'Phi_AOU_{cc}_{layer}'].isel(Y=range(0,54)).values    # 54,10,nlat,nlon
    Phi_O2sol = Phi_O2 - Phi_AOU                                         # 54,10,nlat,nlon
    #
    for xx in range(a1-a1, a2-a1):
        for yy in range(b1-b1, b2-b1):
            for lead in range(10):
                Phi_0 = Phi_ori[:, lead, xx, yy]
                Phi_1 = Phi_bar[lead, xx, yy]
                Phi_2 = Phi_O2[:, lead, xx, yy]
                Phi_3 = Phi_T[:, lead, xx, yy]
                Phi_4 = Phi_S[:, lead, xx, yy]
                Phi_5 = Phi_AOU[:, lead, xx, yy]
                Phi_6 = Phi_O2sol[:, lead, xx, yy]
                Phi_7 = Phi_0-(Phi_1+Phi_2+Phi_3+Phi_4)
                #
                var_0 = 1./(len(Phi_0)-1.)*np.sum((Phi_0 - np.mean(Phi_0))**2)
                var_2 = 1./(len(Phi_0)-1.)*np.sum((Phi_2 - np.mean(Phi_2))**2)
                var_3 = 1./(len(Phi_0)-1.)*np.sum((Phi_3 - np.mean(Phi_3))**2)
                var_4 = 1./(len(Phi_0)-1.)*np.sum((Phi_4 - np.mean(Phi_4))**2)
                var_5 = 1./(len(Phi_0)-1.)*np.sum((Phi_5 - np.mean(Phi_5))**2)
                var_6 = 1./(len(Phi_0)-1.)*np.sum((Phi_6 - np.mean(Phi_6))**2)
                var_7 = 1./(len(Phi_0)-1.)*np.sum((Phi_7 - np.mean(Phi_7))**2)
                # co-variance
                var_23 = 2./(len(Phi_0)-1.)*np.sum((Phi_2 - np.mean(Phi_2))*(Phi_3 - np.mean(Phi_3)))
                var_24 = 2./(len(Phi_0)-1.)*np.sum((Phi_2 - np.mean(Phi_2))*(Phi_4 - np.mean(Phi_4)))
                #var_25 = 2./(len(Phi_0)-1.)*np.sum((Phi_2 - np.mean(Phi_2))*(Phi_5 - np.mean(Phi_5)))
                #var_26 = 2./(len(Phi_0)-1.)*np.sum((Phi_2 - np.mean(Phi_2))*(Phi_6 - np.mean(Phi_6)))
                var_27 = 2./(len(Phi_0)-1.)*np.sum((Phi_2 - np.mean(Phi_2))*(Phi_7 - np.mean(Phi_7)))
                #
                var_34 = 2./(len(Phi_0)-1.)*np.sum((Phi_3 - np.mean(Phi_3))*(Phi_4 - np.mean(Phi_4)))
                var_35 = 2./(len(Phi_0)-1.)*np.sum((Phi_3 - np.mean(Phi_3))*(Phi_5 - np.mean(Phi_5)))
                var_36 = 2./(len(Phi_0)-1.)*np.sum((Phi_3 - np.mean(Phi_3))*(Phi_6 - np.mean(Phi_6)))
                var_37 = 2./(len(Phi_0)-1.)*np.sum((Phi_3 - np.mean(Phi_3))*(Phi_7 - np.mean(Phi_7)))
                #
                var_45 = 2./(len(Phi_0)-1.)*np.sum((Phi_4 - np.mean(Phi_4))*(Phi_5 - np.mean(Phi_5)))
                var_46 = 2./(len(Phi_0)-1.)*np.sum((Phi_4 - np.mean(Phi_4))*(Phi_6 - np.mean(Phi_6)))
                var_47 = 2./(len(Phi_0)-1.)*np.sum((Phi_4 - np.mean(Phi_4))*(Phi_7 - np.mean(Phi_7)))
                #
                var_56 = 2./(len(Phi_0)-1.)*np.sum((Phi_5 - np.mean(Phi_5))*(Phi_6 - np.mean(Phi_6)))
                var_57 = 2./(len(Phi_0)-1.)*np.sum((Phi_5 - np.mean(Phi_5))*(Phi_7 - np.mean(Phi_7)))
                #
                var_67 = 2./(len(Phi_0)-1.)*np.sum((Phi_6 - np.mean(Phi_6))*(Phi_7 - np.mean(Phi_7)))
                #
                data = np.array([var_0,var_2,var_3,var_4,var_5,var_6,var_7,var_23,var_24,var_27,var_34,var_35,var_36,var_37,var_45,var_46,var_47,var_56,var_57,var_67])
                terms = ["Phi'",'O2','T','S','AOU','O2sol','Resi','O2-T','O2-S','O2-Resi','T-S','AOU-T','O2sol-T','T-Resi','AOU-S','O2sol-S','S-Resi','AOU-O2sol','AOU-Resi','O2sol-Resi']
                variance[cc, :, lead, xx, yy] = data.copy()

In [12]:
data = xr.Dataset({"variance": (("nphi", "nterms", "nlead", "nlat", "nlon"), variance)})

In [13]:
dout = f'/glade/scratch/{USER}/DPLE-results'
data.load()

<xarray.Dataset>
Dimensions:   (nphi: 3, nterms: 20, nlead: 10, nlat: 140, nlon: 195)
Dimensions without coordinates: nphi, nterms, nlead, nlat, nlon
Data variables:
    variance  (nphi, nterms, nlead, nlat, nlon) float64 nan nan nan ... nan nan

In [14]:
%%time
data.to_netcdf(f'{dout}/DPLE_driftcorrected_spatial_distribution_of_variance_Phi_components_{layer}.nc', mode='w')

CPU times: user 6.81 ms, sys: 39.9 ms, total: 46.7 ms
Wall time: 112 ms
